# 22. 搜索二维矩阵 II 

编写一个高效的算法来搜索 `_m_ x _n_` 矩阵 `matrix` 中的一个目标值 `target` 。该矩阵具有以下特性：

*   每行的元素从左到右升序排列。
*   每列的元素从上到下升序排列。

**示例 1：** 

![](https://assets.leetcode-cn.com/aliyun-lc-upload/uploads/2020/11/25/searchgrid2.jpg)
> **输入：** matrix = \[\[1,4,7,11,15],\[2,5,8,12,19],\[3,6,9,16,22],\[10,13,14,17,24],\[18,21,23,26,30]], target = 5
> **输出：** true
> 

**示例 2：** 

![](https://assets.leetcode-cn.com/aliyun-lc-upload/uploads/2020/11/25/searchgrid.jpg)
> **输入：** matrix = \[\[1,4,7,11,15],\[2,5,8,12,19],\[3,6,9,16,22],\[10,13,14,17,24],\[18,21,23,26,30]], target = 20
> **输出：** false
> 

**提示：** 

*   `m == matrix.length`
*   `n == matrix[i].length`
*   `1 <= n, m <= 300`
*   `-10^9 <= matrix[i][j] <= 10^9`
*   每行的所有元素从左到右升序排列
*   每列的所有元素从上到下升序排列
*   `-10^9 <= target <= 10^9`

In [ ]:
# 贪心算法（未尝试是否可行）
# 时间复杂度O(m*n)过高，并且not complete，不适用于这一问题
class Solution(object):
    def searchMatrix(self, matrix, target):
        """
        :type matrix: List[List[int]]
        :type target: int
        :rtype: bool
        """
        # 要求搜索高效性
        # 遍历从(0,0)元素开始
        # 考虑到排列的升序特性，一个高效的搜索方案应该有：
        # 1：确认左、下侧的元素是否大于target
        # 2：若大于target，则不考虑
        # 3：若小于target，则移动到其中更大的对象
        row = len(matrix)
        col = len(matrix[0])
        i = 0
        j = 0
        flag = False
        loop = True
        while loop:
            # 找到后停止循环
            if matrix[i][j] == target:
                flag = True
                loop = False
            # 若无匹配对象，停止循环
            if  (matrix[i + 1][j] > target and matrix[i][j + 1] > target) or \
                (i == row - 1 and matrix[i][j + 1] > target) or \
                (j == col - 1 and matrix[i + 1][j] > target) or \
                (i == row - 1 and j == row - 1):
                loop = False
            # 比较下端与右端元素
            if i < row - 1 and j < col - 1:
                if matrix[i + 1][j] <= target and matrix[i][j + 1] <= target:
                    if matrix[i + 1][j] > matrix[i][j + 1]:
                        i += 1
                    else:
                        j += 1
            elif i == row - 1:
                if matrix[i][j + 1] <= target:
                    j += 1
            elif j == col - 1:
                if matrix[i + 1][j] <= target:
                    i += 1



其他思路以及gemini的答复

您好，这是一个很有趣的思路，我们来深入探讨一下“从左上开始，沿对角线向右下移动”的可行性。

总的来说，这个思路**同样存在缺陷**，虽然它看起来似乎利用了矩阵的有序性，但在关键的决策步骤上，它无法有效地缩小搜索范围。

### 对角线思路的分析

我们来模拟一下这个策略。假设我们从 `(0,0)` 开始，沿着主对角线 `(i, i)` 移动。

在对角线上的任意一点 `matrix[i][i]`，我们将其与 `target` 比较：

1.  **如果 `matrix[i][i] == target`**

      * 太棒了，我们找到了目标，可以直接返回 `True`。

2.  **如果 `matrix[i][i] > target`**

      * 因为 `matrix[i][i]` 右边和下边的所有元素都比它大，所以我们可以确定 `target` **不可能**存在于以 `(i,i)` 为左上角的右下角子矩阵中。
      * **但问题在于**，`target` 仍然可能存在于 `(i,i)` 上方的区域，或者左方的区域。此时，我们该如何继续搜索剩下的区域呢？问题变得复杂了。

3.  **如果 `matrix[i][i] < target`**

      * 我们知道 `target` 一定在 `matrix[i][i]` 的右边或下边（或者更右下的位置）。
      * **这正是问题的症结所在！** 我们无法做出一个唯一的、确定的选择。`target` 是在右边的子矩阵里，还是在下边的子矩阵里？我们不知道。
      * 为了找到它，我们可能需要同时搜索两个区域，这通常意味着需要使用递归或者栈（深度优先/广度优先搜索），算法的复杂度会大大增加，从根本上失去了线性时间 `O(m+n)` 的优势。

### 举例说明决策的困境

假设 `target = 13`，矩阵如下：

```
[ [ 1,   4,   7,  11,  15],
  [ 2,   5,   8,  12,  19],
  [ 3,   6,   9,  16,  22],
  [10,  13,  14,  17,  24],  // <-- 目标在这里
  [18,  21,  23,  26,  30] ]
```

1.  从 `(0,0)` 开始，`matrix[0][0] = 1`。因为 `1 < 13`，我们继续沿对角线移动。
2.  来到 `(1,1)`，`matrix[1][1] = 5`。因为 `5 < 13`，继续移动。
3.  来到 `(2,2)`，`matrix[2][2] = 9`。因为 `9 < 13`，继续移动。
4.  来到 `(3,3)`，`matrix[3][3] = 17`。**现在 `17 > 13`。**

根据我们刚才的分析，我们知道 `target` 不可能在以 `(3,3)` 为左上角的右下角区域。那么 `target` 只能在下图的灰色区域内：

我们已经排除了右下角的 2x2 矩阵，但留下的搜索区域非常不规则。我们现在应该从哪里继续搜索呢？是从 `(2,3)` 开始？还是从 `(3,2)` 开始？或者需要把所有剩下的灰色区域都检查一遍？

这个算法没有一个清晰、简单的下一步。任何后续的搜索都会退化成更复杂的子问题。

### 小结

与左上同理的，从右下开始依然无法确认出一个规则区域。

如果要排除得到一个规则的搜索区域，需要考虑右上角或左下角开始的算法。

In [ ]:
class Solution(object):
    def searchMatrix(self, matrix, target):
        """
        :type matrix: List[List[int]]
        :type target: int
        :rtype: bool
        """
        if not matrix or not matrix[0]:
            return False

        rows = len(matrix)
        cols = len(matrix[0])

        # 从右上角开始
        row = 0
        col = cols - 1

        while row < rows and col >= 0:
            current_element = matrix[row][col]
            
            if current_element == target:
                return True
            elif current_element > target:
                # 元素太大，排除此列
                col -= 1
            else: # current_element < target
                # 元素太小，排除此行
                row += 1
        
        return False